In [22]:
import os
import random
import time
import datetime

import openai

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.sdk.data_types.trace_tree import Trace

In [23]:
# get openai API key
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

PROJECT = "dlai_llm"
WANDB_API_KEY = os.environ['WANDB_API_KEY']

MODEL_NAME = "gpt-3.5-turbo"
openai.api_key  = os.environ['OPENAI_API_KEY']


In [24]:
wandb.login(
    anonymous="allow",
	key=WANDB_API_KEY,
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [25]:
run = wandb.init(
    project=PROJECT, 
    job_type="generation",
)

In [26]:
from openai import OpenAI
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [27]:
from openai.types.chat import ChatCompletion

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs) -> ChatCompletion:
    return openai.chat.completions.create(**kwargs)


In [28]:
from wandb import Table

def generate_and_print(
        system_prompt:str, 
        user_prompt:str, 
        table:Table,
        n:int = 5,
    ) -> None:
    
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    
    start_time = time.time()
    responses = completion_with_backoff(
		n = n,
		model=MODEL_NAME,
		messages=messages,
	)
    elapsed_time = time.time() - start_time
    
    for response in responses.choices:
        generation = response.message.content
        print(generation)
        
    table.add_data(
		system_prompt,
		user_prompt,
		[response.message.content for response in responses.choices],
		elapsed_time,
		datetime.datetime.fromtimestamp(responses.created),
		responses.model,
		responses.usage.prompt_tokens,
		responses.usage.completion_tokens,
		responses.usage.total_tokens
	)
    

In [29]:
system_prompt = """You are a creative copywriter.
You're given a category of game asset, \
and your goal is to design a name of that asset.
The game is set in a fantasy world \
where everyone laughs and respects each other, 
while celebrating diversity."""



In [30]:
# Define W&B Table to store generations
columns = ["system_prompt", "user_prompt", "generations", "elapsed_time", "timestamp",\
            "model", "prompt_tokens", "completion_tokens", "total_tokens"]
table = wandb.Table(columns=columns)


In [31]:
user_prompt = "A new The Big Bang Theory character"
generate_and_print(system_prompt, user_prompt, table)


Cosmic Jester of Infinite Laughter
Name: Harmony Quantumflux
Name: Amara Spectra
Description: Amara Spectra is a quirky and brilliant astrophysicist from the planet Galactica. With her radiant energy and love for all things cosmic, she brings a fresh perspective and a whole lot of laughter to the group. Her passion for learning and her kind heart make her an instant hit with the gang, sparking new adventures and discoveries in every episode.
Name: Harmony Gigglesnort
Name: Harmony Hilarity
Description: A quirky and lovable character who brings a harmonious blend of laughter and wit to The Big Bang Theory group. With her infectious humor and respect for diversity, Harmony Hilarity adds a fresh and entertaining dynamic to the show's already colorful cast of characters.


In [32]:
user_prompt = "jewel"
generate_and_print(system_prompt, user_prompt, table)


Harmony Gems
Gleaming Harmony Gems
Harmony Gems
Harmony Gems
Sparkle Harmony Gems


In [33]:
wandb.log({"simple_generations": table})
run.finish()

## 2. Using Tracer to log more complex chains

How can we get more creative outputs? Let's design an LLM chain that will first randomly pick a fantasy world, and then generate character names. We will demonstrate how to use Tracer in such scenario. We will log the inputs and outputs, start and end times, whether the OpenAI call was successful, the token usage, and additional metadata.

In [34]:
worlds = [
    "a mystic medieval island inhabited by intelligent and funny frogs",
    "a modern castle sitting on top of a volcano in a faraway galaxy",
    "a digital world inhabited by friendly machine learning engineers"
]

In [35]:
# define your config
model_name = "gpt-3.5-turbo"
temperature = 0.7
system_message = """You are a creative copywriter. 
You're given a category of game asset and a fantasy world.
Your goal is to design a name of that asset.
Provide the resulting name only, no additional description.
Single name, max 3 words output, remember!"""


In [42]:
def run_creative_chain(query:str) -> None:
    # part 1 - a chain is started...
    start_time_ms = round(datetime.datetime.now().timestamp() * 1000)

    root_span = Trace(
          name="MyCreativeChain",
          kind="chain",
          start_time_ms=start_time_ms,
          metadata={"user": "student_1"},
          model_dict={"_kind": "CreativeChain"}
          )

    # part 2 - your chain picks a fantasy world
    time.sleep(3)
    world = random.choice(worlds)
    expanded_prompt = f'Game asset category: {query}; fantasy world description: {world}'
    tool_end_time_ms = round(datetime.datetime.now().timestamp() * 1000)

    # create a Tool span 
    tool_span = Trace(
          name="WorldPicker",
          kind="tool",
          status_code="success",
          start_time_ms=start_time_ms,
          end_time_ms=tool_end_time_ms,
          inputs={"input": query},
          outputs={"result": expanded_prompt},
          model_dict={"_kind": "tool", "num_worlds": len(worlds)}
          )

    # add the TOOL span as a child of the root
    root_span.add_child(tool_span)

    # part 3 - the LLMChain calls an OpenAI LLM...
    messages=[
      {"role": "system", "content": system_message},
      {"role": "user", "content": expanded_prompt}
    ]

    response = completion_with_backoff(model=model_name,
                                       messages=messages,
                                       max_tokens=12,
                                       temperature=temperature)   

    llm_end_time_ms = round(datetime.datetime.now().timestamp() * 1000)
    response_text = response.choices[0].message.content #["choices"][0]["message"]["content"]
    token_usage = response.usage.to_dict() #["usage"].to_dict()

    llm_span = Trace(
          name="OpenAI",
          kind="llm",
          status_code="success",
          metadata={"temperature":temperature,
                    "token_usage": token_usage, 
                    "model_name":model_name},
          start_time_ms=tool_end_time_ms,
          end_time_ms=llm_end_time_ms,
          inputs={"system_prompt":system_message, "query":expanded_prompt},
          outputs={"response": response_text},
          model_dict={
              "_kind": "Openai", 
              "engine": response.model, #["model"], 
              "model": response.object, #["object"]
            }
          )

    # add the LLM span as a child of the Chain span...
    root_span.add_child(llm_span)

    # update the end time of the Chain span
    root_span.add_inputs_and_outputs(
          inputs={"query":query},
          outputs={"response": response_text})

    # update the Chain span's end time
    root_span.end_time_ms = llm_end_time_ms


    # part 4 - log all spans to W&B by logging the root span
    root_span.log(name="creative_trace")
    print(f"Result: {response_text}")


In [43]:
# Let's start a new wandb run
wandb.init(project=PROJECT, job_type="generation")


In [44]:
run_creative_chain("hero")

Result: Pyro Knight


In [45]:
run_creative_chain("jewel")

Result: Volcano Emberstone


In [46]:
wandb.finish()

## Langchain agent

In the third scenario, we'll introduce an agent that will use tools such as WorldPicker and NameValidator to come up with the ultimate name. We will also use Langchain here and demonstrate its W&B integration.


In [49]:
# Import things that are needed generically
from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool

from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


In [50]:
wandb.init(project=PROJECT, job_type="generation")

In [51]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

In [56]:
from typing import List

class WorldPickerTool(BaseTool):
    name: str = "pick_world"
    description: str = "pick a virtual game world for your character or item naming"
    worlds: List[str] = [
                "a mystic medieval island inhabited by intelligent and funny frogs",
                "a modern anthill featuring a cyber-ant queen and her cyber-ant-workers",
                "a digital world inhabited by friendly machine learning engineers"
            ]

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        time.sleep(1)
        return random.choice(self.worlds)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("pick_world does not support async")
        
class NameValidatorTool(BaseTool):
    
    name: str = "validate_name"
    description: str = "validate if the name is properly generated"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        time.sleep(1)
        if len(query) < 20:
            return f"This is a correct name: {query}"
        else:
            return f"This name is too long. It should be shorter than 20 characters."

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("validate_name does not support async")

In [57]:
llm = ChatOpenAI(temperature=0.7)

In [58]:
tools = [WorldPickerTool(), NameValidatorTool()]
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

/var/folders/j3/qq7z88g96rl91lrdbj227_r80000gn/T/ipykernel_78233/2004885880.py:2: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [59]:
agent.run(
    "Find a virtual game world for me and imagine the name of a hero in that world"
)

/var/folders/j3/qq7z88g96rl91lrdbj227_r80000gn/T/ipykernel_78233/1240652710.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run(




> Entering new AgentExecutor chain...
I need to pick a virtual game world first and then come up with a hero's name in that world.
Action: pick_world
Action Input: None
Observation: a digital world inhabited by friendly machine learning engineers
Thought:Now I need to come up with a name for a hero in this virtual world.
Action: validate_name
Action Input: BraveBot
Observation: This is a correct name: BraveBot
Thought:I have a virtual game world called Friendly Machine Learning Engineers and a hero named BraveBot.
Final Answer: BraveBot in the virtual world of Friendly Machine Learning Engineers.

> Finished chain.


'BraveBot in the virtual world of Friendly Machine Learning Engineers.'

In [60]:
agent.run(
    "Find a virtual game world for me and imagine the name of a jewel in that world"
)



> Entering new AgentExecutor chain...
I should pick a virtual game world first and then come up with a name for a jewel in that world.
Action: pick_world
Action Input: fantasy RPG world
Observation: a modern anthill featuring a cyber-ant queen and her cyber-ant-workers
Thought:This world has a futuristic twist, so the jewel name should reflect that.
Action: validate_name
Action Input: Cyber Shard
Observation: This is a correct name: Cyber Shard
Thought:I now have the final name for a jewel in the fantasy RPG world.
Final Answer: Cyber Shard

> Finished chain.


'Cyber Shard'

In [61]:
agent.run(
    "Find a virtual game world for me and imagine the name of food in that world."
)



> Entering new AgentExecutor chain...
I need to pick a virtual game world first before I can imagine the name of food in it.
Action: pick_world
Action Input: N/A
Observation: a digital world inhabited by friendly machine learning engineers
Thought:Now that I have a virtual game world, I can imagine the name of food in it.
Action: validate_name
Action Input: ByteBites
Observation: This is a correct name: ByteBites
Thought:I now know the final answer
Final Answer: ByteBites in the digital world inhabited by friendly machine learning engineers.

> Finished chain.


'ByteBites in the digital world inhabited by friendly machine learning engineers.'

In [62]:
wandb.finish()